In [ ]:
import openai
import time
import json
import yfinance as yf

Function that gets the stock price

In [ ]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

Define the function for Open-AI

In [ ]:
tools_list = [{
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Retrieve the latest closing price of a stock using its ticker symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The ticker symbol of the stock"
                }
            },
            "required": ["symbol"]
        }
    }
}]

Initialize open ai

In [ ]:
client = openai.OpenAI()

Step 1: Create an Assistant

In [ ]:
assistant = client.beta.assistants.create(
    name="Data Analyst Assistant",
    instructions="You are a personal Data Analyst",
    model="gpt-4-1106-preview",
    tools=tools_list
)

Step 2: Create a Thread

In [ ]:
thread = client.beta.threads.create()

Step 3: Add a Message to a Thread

In [ ]:
messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can you please provide me stock price of Apple?"
)

Step 4: Run the Assistant

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Raul."
)

print(run.model_dump_json(indent=4))

# Get the status of the run
while True:
    # Wait for 5 seconds
    time.sleep(5)

    # Retrieve run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run_status.model_dump_json(indent=4))

    # If the run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        # Loop through messages and print content based on the role
        for msg in messages.data:
            role = msg.role
            content = msg.content[0].text.value
            print(f"{role.capitalize()}: {content}")
        break
    elif run_status.status == 'requieres_action':
        print("Requires action")
        # Parse the data
        required_actions = run_status.required_action.submit_tool_outputs.model_dump()
        print(required_actions)
        tools_output = []
        for action in required_actions["tool_calls"]:
            # Get function
            func_name = action["function"]["name"]
            # Get arguments
            arguments = json.load(action["function"]["arguments"])
            if func_name == "get_stock_price":
                output = get_stock_price(arguments["symbol"])
                tools_output.append({
                    "tool_call_id": action["id"],
                    "output": output
                })
            else:
                raise ValueError(f"Unknown function: {func_name}")
            
            print("Submitting outputs back to the Assistant...")
            # Submit the tool outputs to Assistant API
            client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_outputs=tools_output
            )

    else:
        print("Waiting for the Assistant to process...")
        time.sleep(5)